In [1]:
import csv
import operator
import time
import random
import math
import functools
import seaborn
import scipy.io
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
from os import listdir
from os.path import dirname, join

In [21]:
def find_stationary_location(A):
    res = 1
    A = A[::-1]
    for i in range(len(A)):
        if A[i] == 0:
            return len(A)-i+1
    return res

## Reading in FMRI files

In [3]:
files = listdir('E:/projects/FMRI_820subjects')
print("Total number of files loaded: "+str(len(files)))

# construct a dictionary to relate numbering back to original files
filename_to_num = {}
for i in range(len(files)):
    filename_to_num[i] = files[i]

Total number of files loaded: 820


In [5]:
# loop through all files to extract ROI time-series from each subject and store in the tensor matrix subjects[]
subjects = []
# define file directory
input_files_dir = join('E:','/projects','FMRI_820subjects')
output_files_dir = join('E:','/projects','/connectome','results')
for file in files:
    file_name = join(input_files_dir,file)
    temp = scipy.io.loadmat(file_name)  
    subjects.append(temp['tc'])


## Creating Files for Storage of Results

In [62]:
# create two big .csv's one for positive time and one for negative time, each stores the time needed for the regions to reach 
# their final stationarity for all subjects
df = pd.DataFrame(list())
region_nums = [None]
for i in range(1,161):
    region_nums.append('region '+str(i))
df.to_csv('E:/projects/connectome/results/positive_time/positive_times_combined_results.csv')
df.to_csv('E:/projects/connectome/results/negative_time/negative_times_combined_results.csv')
with open('E:/projects/connectome/results/positive_time/positive_times_combined_results.csv', 'w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(region_nums)
with open('E:/projects/connectome/results/negative_time/negative_times_combined_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(region_nums)

In [63]:
# create files to store results for each subject within regions at different time steps
time_steps = [None]
for i in range(1, 121):
    step_num ='step ' + str(i)
    time_steps.append(step_num)

for i in range(len(subjects)):
    df = pd.DataFrame(list())
    sub_file = 'E:/projects/connectome/results/positive_time/'+'sub_'+str(i+1)+'_positive_time.csv'
    df.to_csv(sub_file)
    with open(sub_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(time_steps)
for i in range(len(subjects)):
    df = pd.DataFrame(list())
    sub_file = 'E:/projects/connectome/results/negative_time/'+'sub_'+str(i+1)+'_negative_time.csv'
    df.to_csv(sub_file)
    with open(sub_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(time_steps)

In [64]:
# finding how long it takes for the time series to stationarize for each region of each subject in both positive and negative
# time directions. 
# The increment is 10 time points.
increment = 10
for i in range(len(subjects)): 
    start_time = time.time()
    positive_times_for_each_subject = ['subject '+str(i+1)]
    negative_times_for_each_subject = ['subject '+str(i+1)]
    pos_time_sub_file = 'E:/projects/connectome/results/positive_time/'+'sub_'+str(i+1)+'_positive_time.csv'
    neg_time_sub_file = 'E:/projects/connectome/results/negative_time/'+'sub_'+str(i+1)+'_negative_time.csv'
    
    for region in range(160):
        pos_time_region_res = ['region_'+str(region+1)]
        neg_time_region_res = ['region_'+str(region+1)]
        for step in range(len(subjects[0][:,0])//increment):
            end = 10*(step+1)
            ts = subjects[i][:,region][:end]
            reversed_ts = subjects[i][:,region][-end:]
            ADF_test = adfuller(ts)
            reversed_ADF_test = adfuller(reversed_ts)
            if ADF_test[4]['1%'] > ADF_test[0]:
                pos_time_region_res.append(1)
            else:
                pos_time_region_res.append(0)
            if reversed_ADF_test[4]['1%'] > reversed_ADF_test[0]:
                neg_time_region_res.append(1)
            else:
                neg_time_region_res.append(0)
        with open(pos_time_sub_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(pos_time_region_res)
        with open(neg_time_sub_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(neg_time_region_res)
        
        
        # skip the first entry because this entry is actually 'region_num'
        positive_times_for_each_subject.append(find_stationary_location(pos_time_region_res[1:])*10)
        negative_times_for_each_subject.append(find_stationary_location(neg_time_region_res[1:])*10)
    
    # write to large_data_set_result.csv for result storage after each subject is processed
    with open('E:/projects/connectome/results/positive_time/positive_times_combined_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(positive_times_for_each_subject)
    with open('E:/projects/connectome/results/negative_time/negative_times_combined_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(negative_times_for_each_subject)
    print("subject "+str(i) +" completed"+", %s seconds taken" % (time.time() - start_time))

subject 0 completed, 389.1613805294037 seconds taken
subject 1 completed, 389.2807140350342 seconds taken
subject 2 completed, 386.82291173934937 seconds taken
subject 3 completed, 385.5043740272522 seconds taken
subject 4 completed, 381.2720642089844 seconds taken
subject 5 completed, 391.8977761268616 seconds taken
subject 6 completed, 384.0817952156067 seconds taken
subject 7 completed, 394.90519881248474 seconds taken
subject 8 completed, 402.8086516857147 seconds taken
subject 9 completed, 417.12638783454895 seconds taken
subject 10 completed, 403.8723769187927 seconds taken
subject 11 completed, 426.22848987579346 seconds taken
subject 12 completed, 435.14026045799255 seconds taken
subject 13 completed, 385.7294428348541 seconds taken
subject 14 completed, 391.93221378326416 seconds taken
subject 15 completed, 380.55209517478943 seconds taken
subject 16 completed, 383.5803020000458 seconds taken
subject 17 completed, 384.45870089530945 seconds taken
subject 18 completed, 387.3900

subject 150 completed, 387.4423305988312 seconds taken
subject 151 completed, 387.3047139644623 seconds taken
subject 152 completed, 388.82727241516113 seconds taken
subject 153 completed, 389.80229783058167 seconds taken
subject 154 completed, 388.75448656082153 seconds taken
subject 155 completed, 386.2682044506073 seconds taken
subject 156 completed, 384.3972125053406 seconds taken
subject 157 completed, 384.8939964771271 seconds taken
subject 158 completed, 382.2841753959656 seconds taken
subject 159 completed, 387.3984100818634 seconds taken
subject 160 completed, 387.8932418823242 seconds taken
subject 161 completed, 387.7380576133728 seconds taken
subject 162 completed, 387.8424913883209 seconds taken
subject 163 completed, 389.27316999435425 seconds taken
subject 164 completed, 385.4630026817322 seconds taken
subject 165 completed, 387.5965383052826 seconds taken
subject 166 completed, 387.1064500808716 seconds taken
subject 167 completed, 386.77377104759216 seconds taken
subje

subject 299 completed, 381.839120388031 seconds taken
subject 300 completed, 380.42489743232727 seconds taken
subject 301 completed, 381.6341598033905 seconds taken
subject 302 completed, 382.5486707687378 seconds taken
subject 303 completed, 381.72592639923096 seconds taken
subject 304 completed, 378.8214330673218 seconds taken
subject 305 completed, 379.3532497882843 seconds taken
subject 306 completed, 380.4844331741333 seconds taken
subject 307 completed, 385.99195885658264 seconds taken
subject 308 completed, 383.14305448532104 seconds taken
subject 309 completed, 380.0237612724304 seconds taken
subject 310 completed, 382.9537105560303 seconds taken
subject 311 completed, 384.2986385822296 seconds taken
subject 312 completed, 386.4136154651642 seconds taken
subject 313 completed, 383.4212143421173 seconds taken
subject 314 completed, 382.05188965797424 seconds taken
subject 315 completed, 381.67710304260254 seconds taken
subject 316 completed, 385.90415596961975 seconds taken
subj

subject 447 completed, 443.5069456100464 seconds taken
subject 448 completed, 443.65525794029236 seconds taken
subject 449 completed, 445.32544207572937 seconds taken
subject 450 completed, 406.71178913116455 seconds taken
subject 451 completed, 389.04916620254517 seconds taken
subject 452 completed, 388.3617904186249 seconds taken
subject 453 completed, 388.3051142692566 seconds taken
subject 454 completed, 389.1372973918915 seconds taken
subject 455 completed, 392.0074164867401 seconds taken
subject 456 completed, 389.50251054763794 seconds taken
subject 457 completed, 442.3489019870758 seconds taken
subject 458 completed, 423.8197195529938 seconds taken
subject 459 completed, 391.35103273391724 seconds taken
subject 460 completed, 412.0062584877014 seconds taken
subject 461 completed, 444.8066625595093 seconds taken
subject 462 completed, 450.9652318954468 seconds taken
subject 463 completed, 450.53224658966064 seconds taken
subject 464 completed, 454.17681789398193 seconds taken
su

subject 595 completed, 384.4421079158783 seconds taken
subject 596 completed, 383.1957585811615 seconds taken
subject 597 completed, 401.8583688735962 seconds taken
subject 598 completed, 389.4772882461548 seconds taken
subject 599 completed, 388.0709080696106 seconds taken
subject 600 completed, 387.448184967041 seconds taken
subject 601 completed, 386.41655564308167 seconds taken
subject 602 completed, 385.78229427337646 seconds taken
subject 603 completed, 382.1885287761688 seconds taken
subject 604 completed, 385.04998660087585 seconds taken
subject 605 completed, 385.28346061706543 seconds taken
subject 606 completed, 385.59382128715515 seconds taken
subject 607 completed, 384.74951696395874 seconds taken
subject 608 completed, 383.09384775161743 seconds taken
subject 609 completed, 386.13937187194824 seconds taken
subject 610 completed, 390.17600560188293 seconds taken
subject 611 completed, 390.4269349575043 seconds taken
subject 612 completed, 383.74524760246277 seconds taken
s

subject 744 completed, 386.59690284729004 seconds taken
subject 745 completed, 386.37556648254395 seconds taken
subject 746 completed, 386.94749879837036 seconds taken
subject 747 completed, 385.38965129852295 seconds taken
subject 748 completed, 386.2135474681854 seconds taken
subject 749 completed, 387.0757141113281 seconds taken
subject 750 completed, 386.22037982940674 seconds taken
subject 751 completed, 388.30414152145386 seconds taken
subject 752 completed, 386.8723464012146 seconds taken
subject 753 completed, 387.79619240760803 seconds taken
subject 754 completed, 388.0532867908478 seconds taken
subject 755 completed, 387.50381803512573 seconds taken
subject 756 completed, 387.7517604827881 seconds taken
subject 757 completed, 388.7201886177063 seconds taken
subject 758 completed, 387.2362759113312 seconds taken
subject 759 completed, 388.50031304359436 seconds taken
subject 760 completed, 388.89461612701416 seconds taken
subject 761 completed, 388.04944610595703 seconds taken

In [66]:
# create two big .csv's one for positive and negative time results both from the middle time point (600).
df = pd.DataFrame(list())
region_nums = [None]
for i in range(1,161):
    region_nums.append('region '+str(i))
df.to_csv('E:/projects/connectome/results/positive_time_from_middle/positive_time_from_middle_combined_results.csv')
df.to_csv('E:/projects/connectome/results/negative_time_from_middle/negative_time_from_middle_combined_results.csv')
with open('E:/projects/connectome/results/positive_time_from_middle/positive_time_from_middle_combined_results.csv', 'w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(region_nums)
with open('E:/projects/connectome/results/negative_time_from_middle/negative_time_from_middle_combined_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(region_nums)

In [67]:
# create files to store results from the mid point
time_steps = [None]
for i in range(1, 61):
    step_num ='step ' + str(i)
    time_steps.append(step_num)

for i in range(len(subjects)):
    df = pd.DataFrame(list())
    sub_file = 'E:/projects/connectome/results/positive_time_from_middle/'+'sub_'+str(i+1)+'_positive_time_from_middle.csv'
    df.to_csv(sub_file)
    with open(sub_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(time_steps)
for i in range(len(subjects)):
    df = pd.DataFrame(list())
    sub_file = 'E:/projects/connectome/results/negative_time_from_middle/'+'sub_'+str(i+1)+'_negative_time_from_middle.csv'
    df.to_csv(sub_file)
    with open(sub_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(time_steps)

In [71]:
a = list(range(100))
a[-1:-10:-1]

[99, 98, 97, 96, 95, 94, 93, 92, 91]

In [73]:
# finding how long it takes for the time series to stationarize for each region of each subject in both positive and negative
# time directions. 
# The increment is 10 time points.
increment = 10
for i in range(len(subjects)): 
    start_time = time.time()
    positive_times_for_each_subject = ['subject '+str(i+1)]
    negative_times_for_each_subject = ['subject '+str(i+1)]
    pos_time_sub_file = 'E:/projects/connectome/results/positive_time_from_middle/'+'sub_'+str(i+1)+'_positive_time_from_middle.csv'
    neg_time_sub_file = 'E:/projects/connectome/results/negative_time_from_middle/'+'sub_'+str(i+1)+'_negative_time_from_middle.csv'
    
    for region in range(160):
        pos_time_region_res = ['region_'+str(region+1)]
        neg_time_region_res = ['region_'+str(region+1)]
        for step in range(len(subjects[0][:,0])//(2*increment)):
            end = 10*(step+1)
            ts = subjects[i][:,region][600-end:600]
            reversed_ts = subjects[i][:,region][600:600+end]
            ADF_test = adfuller(ts)
            reversed_ADF_test = adfuller(reversed_ts)
            if ADF_test[4]['1%'] > ADF_test[0]:
                pos_time_region_res.append(1)
            else:
                pos_time_region_res.append(0)
            if reversed_ADF_test[4]['1%'] > reversed_ADF_test[0]:
                neg_time_region_res.append(1)
            else:
                neg_time_region_res.append(0)
        with open(pos_time_sub_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(pos_time_region_res)
        with open(neg_time_sub_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(neg_time_region_res)
        
        
        # skip the first entry because this entry is actually 'region_num'
        positive_times_for_each_subject.append(find_stationary_location(pos_time_region_res[1:])*10)
        negative_times_for_each_subject.append(find_stationary_location(neg_time_region_res[1:])*10)
    
    # write to large_data_set_result.csv for result storage after each subject is processed
    with open('E:/projects/connectome/results/positive_time_from_middle/positive_time_from_middle_combined_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(positive_times_for_each_subject)
    with open('E:/projects/connectome/results/negative_time_from_middle/negative_time_from_middle_combined_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(negative_times_for_each_subject)
    print("subject "+str(i) +" completed"+", %s seconds taken" % (time.time() - start_time))

subject 0 completed, 128.2892997264862 seconds taken
subject 1 completed, 129.4773509502411 seconds taken
subject 2 completed, 128.93166875839233 seconds taken
subject 3 completed, 130.34275126457214 seconds taken
subject 4 completed, 131.63350176811218 seconds taken
subject 5 completed, 130.95779585838318 seconds taken
subject 6 completed, 130.8785400390625 seconds taken
subject 7 completed, 132.33910536766052 seconds taken
subject 8 completed, 127.39523839950562 seconds taken
subject 9 completed, 126.22696208953857 seconds taken
subject 10 completed, 126.87885451316833 seconds taken
subject 11 completed, 127.41569590568542 seconds taken
subject 12 completed, 127.40496015548706 seconds taken
subject 13 completed, 127.18633723258972 seconds taken
subject 14 completed, 127.84318161010742 seconds taken
subject 15 completed, 127.90665936470032 seconds taken
subject 16 completed, 129.61555552482605 seconds taken
subject 17 completed, 131.01225399971008 seconds taken
subject 18 completed, 1

subject 149 completed, 129.31502985954285 seconds taken
subject 150 completed, 129.87809991836548 seconds taken
subject 151 completed, 129.064190864563 seconds taken
subject 152 completed, 128.58588790893555 seconds taken
subject 153 completed, 128.60250878334045 seconds taken
subject 154 completed, 128.1437909603119 seconds taken
subject 155 completed, 128.5400812625885 seconds taken
subject 156 completed, 128.06668853759766 seconds taken
subject 157 completed, 128.20332622528076 seconds taken
subject 158 completed, 127.98080039024353 seconds taken
subject 159 completed, 128.01206374168396 seconds taken
subject 160 completed, 128.53805923461914 seconds taken
subject 161 completed, 129.37581753730774 seconds taken
subject 162 completed, 129.65463638305664 seconds taken
subject 163 completed, 128.6161892414093 seconds taken
subject 164 completed, 128.50116801261902 seconds taken
subject 165 completed, 128.66399717330933 seconds taken
subject 166 completed, 128.4522099494934 seconds take

subject 297 completed, 129.66443014144897 seconds taken
subject 298 completed, 128.93920707702637 seconds taken
subject 299 completed, 128.47851824760437 seconds taken
subject 300 completed, 128.3936460018158 seconds taken
subject 301 completed, 130.97238492965698 seconds taken
subject 302 completed, 129.6507339477539 seconds taken
subject 303 completed, 129.10027194023132 seconds taken
subject 304 completed, 129.30230259895325 seconds taken
subject 305 completed, 129.3637888431549 seconds taken
subject 306 completed, 129.63609337806702 seconds taken
subject 307 completed, 130.97711634635925 seconds taken
subject 308 completed, 129.67728090286255 seconds taken
subject 309 completed, 129.48378920555115 seconds taken
subject 310 completed, 129.90644574165344 seconds taken
subject 311 completed, 129.48091077804565 seconds taken
subject 312 completed, 130.20611333847046 seconds taken
subject 313 completed, 130.36028504371643 seconds taken
subject 314 completed, 129.63612937927246 seconds t

subject 445 completed, 128.88750171661377 seconds taken
subject 446 completed, 128.45708870887756 seconds taken
subject 447 completed, 128.32044768333435 seconds taken
subject 448 completed, 128.60739064216614 seconds taken
subject 449 completed, 128.63862228393555 seconds taken
subject 450 completed, 128.46391940116882 seconds taken
subject 451 completed, 128.59470224380493 seconds taken
subject 452 completed, 128.90413999557495 seconds taken
subject 453 completed, 128.84260511398315 seconds taken
subject 454 completed, 128.40540409088135 seconds taken
subject 455 completed, 130.80264687538147 seconds taken
subject 456 completed, 129.15010166168213 seconds taken
subject 457 completed, 129.02219009399414 seconds taken
subject 458 completed, 128.23943948745728 seconds taken
subject 459 completed, 128.31361484527588 seconds taken
subject 460 completed, 127.8929591178894 seconds taken
subject 461 completed, 128.25212812423706 seconds taken
subject 462 completed, 128.57811188697815 seconds

subject 593 completed, 128.05009579658508 seconds taken
subject 594 completed, 127.55623888969421 seconds taken
subject 595 completed, 129.53544092178345 seconds taken
subject 596 completed, 128.63862228393555 seconds taken
subject 597 completed, 128.8074688911438 seconds taken
subject 598 completed, 128.71963238716125 seconds taken
subject 599 completed, 129.98159551620483 seconds taken
subject 600 completed, 128.47856044769287 seconds taken
subject 601 completed, 128.65033340454102 seconds taken
subject 602 completed, 128.4561104774475 seconds taken
subject 603 completed, 129.42235207557678 seconds taken
subject 604 completed, 130.44407534599304 seconds taken
subject 605 completed, 128.89530968666077 seconds taken
subject 606 completed, 129.13443064689636 seconds taken
subject 607 completed, 129.64682912826538 seconds taken
subject 608 completed, 128.97830533981323 seconds taken
subject 609 completed, 129.7698357105255 seconds taken
subject 610 completed, 129.91913175582886 seconds t

subject 740 completed, 127.47038578987122 seconds taken
subject 741 completed, 128.06275177001953 seconds taken
subject 742 completed, 128.18380665779114 seconds taken
subject 743 completed, 128.12915062904358 seconds taken
subject 744 completed, 127.58356761932373 seconds taken
subject 745 completed, 129.07001614570618 seconds taken
subject 746 completed, 127.9934868812561 seconds taken
subject 747 completed, 128.18087935447693 seconds taken
subject 748 completed, 128.05790424346924 seconds taken
subject 749 completed, 129.65079164505005 seconds taken
subject 750 completed, 129.9855010509491 seconds taken
subject 751 completed, 128.24041509628296 seconds taken
subject 752 completed, 128.05107235908508 seconds taken
subject 753 completed, 128.8621244430542 seconds taken
subject 754 completed, 127.79926347732544 seconds taken
subject 755 completed, 128.2257752418518 seconds taken
subject 756 completed, 128.4834394454956 seconds taken
subject 757 completed, 128.59567952156067 seconds tak

## Hurst's Exponent

In [6]:
from numpy import log, polyfit, sqrt, std, subtract
from hurst import compute_Hc
def CalcHurstExp(ts):  
    lags = range(2, 100)
    tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags]  
    poly = np.polyfit(np.log(lags), np.log(tau), 1)  
    hurst = poly[0]*2.0  
    return max(hurst,0)

In [9]:
all_H = []

# create one csv for storage of Hurst Exponents
df = pd.DataFrame(list())
region_nums = [None]
for i in range(1,161):
    region_nums.append('region '+str(i))
df.to_csv('E:/projects/connectome/results/HurstExp.csv')
with open('E:/projects/connectome/results/HurstExp.csv', 'w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(region_nums)

In [7]:
for i in range(len(subjects)):
    subject_H = ['sub'+str(i)]
    for region in range(160):
        H, c, data = compute_Hc(subjects[i][:,region])
        subject_H.extend(H)
    with open('E:/projects/connectome/results/HurstExp/HurstExp.csv','w',newline='') as file:
        writer = csv.writer(file)
        writer.writerow(subject_H)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/projects/connectome/results/HurstExp/HurstExp.csv'

In [63]:
region_avg_H=[]
for region in range(0,160):
    region_H  = []
    for i in range(len(subjects)):
        region_H.append(all_H[i][region])
    print("For region "+str(region))
    print(region_H)
    region_avg_H.append(sum(region_H)/float(len(region_H)))
    

For region 0
[0.19859717635076685, 0.13840688432933598, 0.18221809153631585, 0.193785717768027, 0.16035485233313662, 0.2011580924196782, 0.19942420055030785, 0.18936744210105508, 0.20003974008626565, 0.2275004234161657, 0.1750324808609978, 0.21254111586394667, 0.16336185106918974, 0.19608404400906623, 0.14891987297092843, 0.1606252690088529, 0.16591990440870064, 0.20240799666127904, 0.19905249109620352, 0.1892147007238287, 0.24280308525525413, 0.14202673130595242, 0.2171013504440625, 0.1670752702210914, 0.17958666470850423, 0.2271504081065178, 0.1592527816651178, 0.1563625809789788, 0.182199402014509, 0.18139136428431263, 0.13830062119378816, 0.19312368505088945, 0.21380983539636164, 0.25964317980015766, 0.2737255752318297, 0.26721621810151486, 0.16641543382893348, 0.22096253592315057, 0.12342233238817542, 0.1334756817777713, 0.20471430585437608, 0.23772736354180488, 0.22146017836080517, 0.16689903160099376, 0.17346446834144427, 0.18589203507640287, 0.1745181077139304, 0.21344906082007

In [64]:
region_avg_H

[0.189269206791625,
 0.1705591500994482,
 0.14909851777909156,
 0.18325944676925743,
 0.16522351105279745,
 0.18115250280150288,
 0.16997082099445893,
 0.1705649752972992,
 0.18774438483410402,
 0.17735775811514734,
 0.16301090566264265,
 0.17864790128435057,
 0.17170946651828695,
 0.17033179279594504,
 0.156067833699776,
 0.1969876599608777,
 0.1596978131736066,
 0.156594716048649,
 0.19486912217521304,
 0.17572811211465592,
 0.16828661724583227,
 0.155689578939251,
 0.16629024957921026,
 0.16356574801155635,
 0.1650082793690777,
 0.19745150135237236,
 0.18592351269390717,
 0.17755683150118,
 0.15302860670888158,
 0.15437890407156912,
 0.17217949375994965,
 0.17615129219057685,
 0.1789465928292608,
 0.19320241858565795,
 0.19834649397566423,
 0.16950186822149535,
 0.1958764069542021,
 0.1969484403186947,
 0.16890227378276135,
 0.16718245410598165,
 0.18582816629954435,
 0.1558516957158482,
 0.17289281588057837,
 0.17341795707798896,
 0.19155139712504646,
 0.1791804368584668,
 0.179159

## Goodness of Fit of ARMA Models

In [4]:
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARIMAResults
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from datetime import datetime


In [5]:
# models = models[subject][region] stores the opimal models based on AIC
models = [[None for _ in range(len(subjects[0][1]))] for _ in range(len(subjects))]
# MSEs = MSEs[subject][region] stores the mean square error for the corresponding models
MSEs = [[None for _ in range(len(subjects[0][1]))] for _ in range(len(subjects))]
# print(len(models))
# print(len(models[0]))
R2scores= [[None for _ in range(len(subjects[0][1]))] for _ in range(len(subjects))]

In [ ]:
d=0 # take the number of differencing to be 0 at the moment, we can change this later for experimenting purposes

for sub in range(100):
    for region in range(160):
        ts = subjects[sub][:,region]
        order = (0,0,0) 
        prev_R2 = 0 
        for p in range(1,5):
            for q in range(0,5):
                try:
                    model = ARIMA(ts, order=(p,d,q)).fit()
                    predicted = model.predict()
                    cur_R2 = r2_score(ts, predicted)
                    if cur_R2 > prev_R2:
                        order = (p,d,q)
                        prev_R2 = cur_R2
                except:
                    continue
        
        if order != (0,0,0):
            models[sub][region] = model
            R2scores[sub][region] = prev_R2
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
    
    print("subject "+str(sub)+" finished at "+str(current_time))

subject 0 finished at 16:59:51
subject 1 finished at 21:41:32
subject 2 finished at 22:04:01
subject 3 finished at 22:28:27
subject 4 finished at 22:56:03
subject 5 finished at 23:18:59


In [83]:

print("subject "+str(sub)+" finished at:"+str(current_time))


subject 19 finished at:16:30:10
